# Imports

In [2]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

import ollama

from tqdm import tqdm
from collections import defaultdict

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from scipy.spatial.distance import pdist
from sklearn.decomposition import PCA

# Parameters

In [4]:
# List of models
ml_models = [
    ('Logistic Regression', LogisticRegression(random_state=42)),
    ('Random Forest', RandomForestClassifier(random_state=42)),
    ('SVM', SVC(random_state=42)),
    ('KNN', KNeighborsClassifier()),
    ('Gradient Boosting', GradientBoostingClassifier(random_state=42))
]

In [5]:
# Number of rows to synthesize
n_rows = 5000

In [6]:
# Different temperatures for LLM at which data will be synthesized
temperatures = [5, 10, 15]

In [7]:
# CSV file names
base_data_prep_name = "tabular_data_preprocessed_2025_04_03.csv"

In [8]:
# Force CUDA usage
os.environ["OLLAMA_BACKEND"] = "cuda"
os.environ["OLLAMA_NUM_THREADS"] = "16"

# Load Data

In [10]:
df = pd.read_csv(base_data_prep_name)

In [11]:
df.head()

,age,workclass,education_num,occupation,capital_gain,capital_loss,hours_per_week,native_country,income,relationship_not_in_family,...,marital_status_married_civ_spouse,marital_status_married_spouse_absent,marital_status_never_married,marital_status_separated,marital_status_widowed,race_asian_pac_islander,race_black,race_other,race_white,sex_male
0,0.025996,2.137359,1.136512,-1.318460,0.146932,-0.217127,-0.034087,0.289462,0,1.697524,...,-0.919604,-0.114128,1.424944,-0.179829,-0.1791,-0.179161,-0.325728,-0.091554,0.411743,0.70422
1,0.828308,1.454401,1.136512,-0.609318,-0.144804,-0.217127,-2.213032,0.289462,0,-0.589093,...,1.087425,-0.114128,-0.701782,-0.179829,-0.1791,-0.179161,-0.325728,-0.091554,0.411743,0.70422
2,-0.046942,0.088484,-0.419335,-0.136557,-0.144804,-0.217127,-0.034087,0.289462,0,1.697524,...,-0.919604,-0.114128,-0.701782,-0.179829,-0.1791,-0.179161,-0.325728,-0.091554,0.411743,0.70422
3,1.047121,0.088484,-1.197259,-0.136557,-0.144804,-0.217127,-0.034087,0.289462,0,-0.589093,...,1.087425,-0.114128,-0.701782,-0.179829,-0.1791,-0.179161,3.070047,-0.091554,-2.428701,0.70422
4,-0.776316,0.088484,1.136512,0.808965,-0.144804,-0.217127,-0.034087,-4.083380,0,-0.589093,...,1.087425,-0.114128,-0.701782,-0.179829,-0.1791,-0.179161,3.070047,-0.091554,-2.428701,-1.42001


### Proxy meassures for homogeneity of base data

In [13]:
def euclidean(df):
    distances = pdist(df, metric='euclidean')
    avg_distance = np.mean(distances)
    print("Average pairwise distance:", avg_distance)


def pca(df):
    pca = PCA(n_components=3)
    transformed = pca.fit_transform(df)
    explained_variance = pca.explained_variance_ratio_
    print("Explained variance ratio:", explained_variance)
    
    cumulative_variance = np.sum(pca.explained_variance_ratio_)
    print("Cumulative explained variance:", cumulative_variance)

In [14]:
euclidean(df)
pca(df)

Average pairwise distance: 6.391776639787022
Explained variance ratio: [0.12171479 0.08500064 0.07121668]
Cumulative explained variance: 0.2779321020922514


# Experimential setup

In [16]:
# Split the data into training test set
X_train, X_test, y_train, y_test = train_test_split(df.drop('income', axis=1), df['income'], test_size=0.2, random_state=42)

# Print the shapes of the sets
print(f"Training Set: X_train shape = {X_train.shape}, y_train shape = {y_train.shape}")
print(f"Test Set: X_test shape = {X_test.shape}, y_test shape = {y_test.shape}")

Training Set: X_train shape = (39073, 24), y_train shape = (39073,)
Test Set: X_test shape = (9769, 24), y_test shape = (9769,)


# Modeling and Performance metrics

In [18]:
# Function to train and evaluate models with multiple metrics
def evaluate_models_with_metrics(models, X_train, y_train, X_test, y_test):
    results = []
    
    for name, model in models:
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict on test set
        y_test_pred = model.predict(X_test)

        # Store results for the model
        model_results = {
            'Model': name,
            'Test Accuracy': accuracy_score(y_test, y_test_pred),
            'Test Precision': precision_score(y_test, y_test_pred),
            'Test Recall': recall_score(y_test, y_test_pred),
            'Test F1-Score': f1_score(y_test, y_test_pred)
        }
        
        results.append(model_results)

    # Convert results to a pandas DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

In [19]:
# Evaluate models with multiple metrics and print results
results_normal = evaluate_models_with_metrics(ml_models, X_train, y_train, X_test, y_test)
display(results_normal)

,Model,Test Accuracy,Test Precision,Test Recall,Test F1-Score
0,Logistic Regression,0.841744,0.725097,0.553291,0.627649
1,Random Forest,0.847989,0.719033,0.606369,0.657913
2,SVM,0.845634,0.760615,0.524841,0.621106
3,KNN,0.829358,0.672518,0.569427,0.616693
4,Gradient Boosting,0.866414,0.792969,0.603397,0.685315


# Data synthesis experiment

In the case where you have little data you can use LLMs to generate data (with controlled feature variability)

To prove that this works we can set up the following experiment
1. Train a model with 100 rows (small sample size)
2. Train another model with 200 rows where 100 rows from the existing set and then 100 rows generated from the LLM
3. Test first model on test set and validation set
4. Test second model on test set and validation set
5. Compare performance metrics

### Let's quickly make the base DF with our low sample size (100 samples)

In [23]:
# Split the data into training and test sets
X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(df.drop('income', axis=1), df['income'], test_size=0.2, random_state=42)
X_train_base = X_train_base[0:99]
y_train_base = y_train_base[0:99]

# Print the shapes of the sets
print(f"Training Set: X_train shape = {X_train_base.shape}, y_train shape = {y_train_base.shape}")
print(f"Test Set: X_test shape = {X_test_base.shape}, y_test shape = {y_test_base.shape}")

base_results = []
  
for name, ml_model in ml_models:
    # Train the model
    ml_model.fit(X_train_base, y_train_base)
    
    # Predict on test set
    y_test_pred_base = ml_model.predict(X_test_base)

    # Store results for the model
    model_results = {
        'Model': name,
        'Test Accuracy': accuracy_score(y_test_base, y_test_pred_base),
        'Test Precision': precision_score(y_test_base, y_test_pred_base),
        'Test Recall': recall_score(y_test_base, y_test_pred_base),
        'Test F1-Score': f1_score(y_test_base, y_test_pred_base)
    }
    
    base_results.append(model_results)

# Convert results to a pandas DataFrame
base_results = pd.DataFrame(base_results).set_index("Model")
display(base_results)

Training Set: X_train shape = (99, 24), y_train shape = (99,)
Test Set: X_test shape = (9769, 24), y_test shape = (9769,)


,Test Accuracy,Test Precision,Test Recall,Test F1-Score
Model,,,,
Logistic Regression,0.810523,0.636957,0.497665,0.558760
Random Forest,0.813594,0.655413,0.478132,0.552909
SVM,0.803050,0.645510,0.405945,0.498436
KNN,0.791995,0.592338,0.439915,0.504873
Gradient Boosting,0.804893,0.603504,0.555839,0.578691


### Models that we will test

In [25]:
prompt = '''Generate ONE realistic data row where ALL fields align with the variable constraints and logically justify the Target (does the person earn more or less than 50k dollars). 
Return ONLY the pipe-separated row with NO additional text or explanations.

Mandatory Format (pipe-separated, exact order):
Age|Sex|Country|Marital-Status|Occupation|Workclass|Education|Race|Relationship|Hours-Weekly|Capital-Gain|Capital-Loss|Target

Variable Constraints (NO deviations allowed):
Age: Integer (17-90)
Sex: Male or Female
Country: Exact values from: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands
Marital-Status: Exact values from: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse
Occupation: Exact values from: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces
Workclass: Exact values from: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked
Education: Exact values from: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool
Race: Exact values from: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black
Relationship: Exact values from: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried
Hours-Weekly: Integer (1-99)
Capital-Gain: Non-negative integer (0-99999)
Capital-Loss: Non-negative integer (0-99999)
Target: >50K or <50K
'''

In [26]:
def validate_input(input_string):
    # Split the input string by pipe character
    fields = input_string.split('|')
    assert len(fields) == 13, "Input must have exactly 13 fields separated by |"
    
    # Unpack all fields
    age, sex, country, marital_status, occupation, workclass, education, race, relationship, hours_weekly, capital_gain, capital_loss, target = fields
    
    # Validate Age
    age = int(age)
    assert 17 <= age <= 90, "Age must be between 17 and 90"
    
    # Validate Sex
    assert sex in {'Male', 'Female'}, "Sex must be either Male or Female"
    
    # Validate Country
    valid_countries = {
        'United-States', 'Cambodia', 'England', 'Puerto-Rico', 'Canada', 'Germany',
        'Outlying-US(Guam-USVI-etc)', 'India', 'Japan', 'Greece', 'China', 'Cuba',
        'Iran', 'Honduras', 'Philippines', 'Italy', 'Poland', 'Jamaica', 'Vietnam',
        'Mexico', 'Portugal', 'Ireland', 'France', 'Dominican-Republic', 'Laos',
        'Ecuador', 'Haiti', 'Columbia', 'Hungary', 'Guatemala', 'Nicaragua',
        'Scotland', 'Thailand', 'Yugoslavia', 'El-Salvador', 'Trinadad&Tobago',
        'Peru', 'Hong', 'Holand-Netherlands'
    }
    assert country in valid_countries, f"Invalid country: {country}"
    
    # Validate Marital-Status
    valid_marital_statuses = {
        'Married-civ-spouse', 'Divorced', 'Never-married', 'Separated', 'Widowed',
        'Married-spouse-absent', 'Married-AF-spouse'
    }
    assert marital_status in valid_marital_statuses, f"Invalid marital status: {marital_status}"
    
    # Validate Occupation
    valid_occupations = {
        'Tech-support', 'Craft-repair', 'Other-service', 'Sales', 'Exec-managerial',
        'Prof-specialty', 'Handlers-cleaners', 'Machine-op-inspct', 'Adm-clerical',
        'Farming-fishing', 'Transport-moving', 'Priv-house-serv', 'Protective-serv',
        'Armed-Forces'
    }
    assert occupation in valid_occupations, f"Invalid occupation: {occupation}"
    
    # Validate Workclass
    valid_workclasses = {
        'Private', 'Self-emp-not-inc', 'Self-emp-inc', 'Federal-gov', 'Local-gov',
        'State-gov', 'Without-pay', 'Never-worked'
    }
    assert workclass in valid_workclasses, f"Invalid workclass: {workclass}"
    
    # Validate Education
    valid_educations = {
        'Bachelors', 'Some-college', '11th', 'HS-grad', 'Prof-school', 'Assoc-acdm',
        'Assoc-voc', '9th', '7th-8th', '12th', 'Masters', '1st-4th', '10th',
        'Doctorate', '5th-6th', 'Preschool'
    }
    assert education in valid_educations, f"Invalid education: {education}"
    
    # Validate Race
    valid_races = {
        'White', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other', 'Black'
    }
    assert race in valid_races, f"Invalid race: {race}"
    
    # Validate Relationship
    valid_relationships = {
        'Wife', 'Own-child', 'Husband', 'Not-in-family', 'Other-relative', 'Unmarried'
    }
    assert relationship in valid_relationships, f"Invalid relationship: {relationship}"
    
    # Validate Hours-Weekly
    hours_weekly = int(hours_weekly)
    assert 1 <= hours_weekly <= 99, "Hours per week must be between 1 and 99"
    
    # Validate Capital-Gain
    capital_gain = int(capital_gain)
    assert 0 <= capital_gain <= 99999, "Capital gain must be between 0 and 99999"
    
    # Validate Capital-Loss
    capital_loss = int(capital_loss)
    assert 0 <= capital_loss <= 99999, "Capital loss must be between 0 and 99999"
    
    # Validate Target
    assert target in {'>50K', '<50K'}, "Target must be either >50K or <50K"
    
    return True

### Demonstration that "dumb" models have too much error

In [28]:
models = ['llama3.2:1b', 'llama3.2:3b']

In [29]:
# Initialize a dictionary to store results and errors per model
results = {}
error_counts_per_model = defaultdict(lambda: defaultdict(int))

# Iterate through models
for index, model in enumerate(models):
    print("Processing Model: " + model + " (Model " + str(index + 1) + "/" + str(len(models)) + ")")
    results[model] = []
    retries = 0
    valid_samples = 0
    total_samples = 0

    # Loop through the n_rows instances for each model
    for _ in tqdm(range(n_rows), desc=f"Generating Samples for {model}", leave=True):
        total_samples += 1
        try:
            # Generate data
            response = ollama.generate(model=model, prompt=prompt)['response']

            # Check if the response is valid
            validate_input(response)

            # Store the valid response
            results[model].append(response)
            valid_samples += 1
        except Exception as e:
            # Store error per model
            error_counts_per_model[model][str(e)] += 1
            retries += 1

            # Stop early for this model if too many errors (>10% errors)
            if retries >= int(n_rows*0.1):
                print(f"Too many errors for model {model}, stopping early.")
                break  

    # Calculate error rate
    error_rate = (retries / total_samples) * 100
    print(f"Model {model} Error Rate: {error_rate:.2f}%")

    # Print error summary for the current model
    print(f"Error Distribution Summary for Model {model}:")
    for error_msg, count in error_counts_per_model[model].items():
        print(f"  {error_msg}: {count} occurrences")
    print()

Processing Model: llama3.2:1b (Model 1/2)


Generating Samples for llama3.2:1b:  10%|████████████████████▌                                                                                                                                                                                        | 501/5000 [06:04<54:33,  1.37it/s]


Too many errors for model llama3.2:1b, stopping early.
Model llama3.2:1b Error Rate: 99.60%
Error Distribution Summary for Model llama3.2:1b:
  Input must have exactly 13 fields separated by |: 356 occurrences
  Invalid workclass: Other-service: 3 occurrences
  Invalid workclass: Exec-managerial: 7 occurrences
  Age must be between 17 and 90: 56 occurrences
  invalid literal for int() with base 10: '$100000': 1 occurrences
  Sex must be either Male or Female: 16 occurrences
  invalid literal for int() with base 10: '0.00000+': 1 occurrences
  Capital loss must be between 0 and 99999: 17 occurrences
  Target must be either >50K or <50K: 5 occurrences
  Invalid workclass: Prof-specialty: 6 occurrences
  Invalid marital status: Marry-civ-spouse: 8 occurrences
  Invalid race: None: 1 occurrences
  Invalid education: Education:Bachelors: 6 occurrences
  Capital gain must be between 0 and 99999: 6 occurrences
  Invalid education: Education:Prof-specialty: 1 occurrences
  Invalid marital stat

Generating Samples for llama3.2:3b:  13%|██████████████████████████▍                                                                                                                                                                                | 650/5000 [13:20<1:29:18,  1.23s/it]

Too many errors for model llama3.2:3b, stopping early.
Model llama3.2:3b Error Rate: 76.80%
Error Distribution Summary for Model llama3.2:3b:
  Input must have exactly 13 fields separated by |: 187 occurrences
  Sex must be either Male or Female: 280 occurrences
  Invalid marital status: Maiden-head-of-house-hold: 1 occurrences
  Invalid marital status: Maiden-civ-spouse: 2 occurrences
  Age must be between 17 and 90: 7 occurrences
  Capital loss must be between 0 and 99999: 13 occurrences
  Invalid marital status: Maiden-Married-civ-spouse: 1 occurrences
  Capital gain must be between 0 and 99999: 1 occurrences
  Invalid occupation: Farmers-fishers-and-related-occupations: 1 occurrences
  Invalid marital status: Merged-civ-spouse: 1 occurrences
  Target must be either >50K or <50K: 3 occurrences
  Invalid relationship: Married-spouse-absent: 1 occurrences
  Invalid education: Private: 1 occurrences
  Invalid marital status: Maiden-head-of-household: 1 occurrences



### Hence we use smart enough models

In [31]:
models = ['llama3.1', 'mistral']

### Lets generate some data
We will use different temperature and use it as a paramter to see if we can get less homogeneous data

In [33]:
# Initialize a dictionary to store results and errors per model
results = {}
error_counts_per_model = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

# Iterate through models
for index, model in enumerate(models):
    print("Processing Model: " + model + " (Model " + str(index + 1) + "/" + str(len(models)) + ")")
    results[model] = {}

    # Loop through the different temperatures
    for temperature in temperatures:
        print(f"  Using Temperature: {temperature}")
        results[model][temperature] = []
        retries = 0
        valid_samples = 0
        total_samples = 0
        
        # Loop through the x instances for each temperature setting
        for _ in tqdm(range(n_rows), desc=f"Generating Samples for {model} with Temp {temperature}", leave=True):
            total_samples += 1
            try:
                # Generate data with the specified temperature
                response = ollama.generate(model=model, prompt=prompt, options={"temperature": temperature})['response']

                # Check if the response is valid
                validate_input(response)

                # Store the valid response
                results[model][temperature].append(response)
                valid_samples += 1
            except Exception as e:
                # Store error per model and temperature
                error_counts_per_model[model][temperature][str(e)] += 1
                retries += 1

        # Calculate error rate
        error_rate = (retries / total_samples) * 100
        print(f"  Model {model} at Temp {temperature} - Error Rate: {error_rate:.2f}%")

        # Print error summary for the current model and temperature
        print(f"  Error Distribution Summary for Model {model} at Temp {temperature}:")
        for error_msg, count in error_counts_per_model[model][temperature].items():
            print(f"    {error_msg}: {count} occurrences")    
        print()

Processing Model: llama3.1 (Model 1/2)
  Using Temperature: 5


Generating Samples for llama3.1 with Temp 5: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [6:21:19<00:00,  4.58s/it]


  Model llama3.1 at Temp 5 - Error Rate: 26.50%
  Error Distribution Summary for Model llama3.1 at Temp 5:
    Input must have exactly 13 fields separated by |: 251 occurrences
    Capital loss must be between 0 and 99999: 260 occurrences
    invalid literal for int() with base 10: '1231>': 1 occurrences
    Sex must be either Male or Female: 251 occurrences
    Invalid country: MEXICO: 83 occurrences
    invalid literal for int() with base 10: '': 33 occurrences
    invalid literal for int() with base 10: 'Age': 174 occurrences
    Invalid country: MEX: 60 occurrences
    invalid literal for int() with base 10: '1022>': 1 occurrences
    invalid literal for int() with base 10: '1507>': 2 occurrences
    invalid literal for int() with base 10: '1001>': 2 occurrences
    invalid literal for int() with base 10: '1021>': 1 occurrences
    Capital gain must be between 0 and 99999: 17 occurrences
    invalid literal for int() with base 10: '2304>': 1 occurrences
    invalid literal for int(

Generating Samples for llama3.1 with Temp 10: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [7:53:11<00:00,  5.68s/it]


  Model llama3.1 at Temp 10 - Error Rate: 27.28%
  Error Distribution Summary for Model llama3.1 at Temp 10:
    Capital loss must be between 0 and 99999: 295 occurrences
    invalid literal for int() with base 10: 'Age': 121 occurrences
    Sex must be either Male or Female: 215 occurrences
    Input must have exactly 13 fields separated by |: 253 occurrences
    Invalid country: MEX: 68 occurrences
    Capital gain must be between 0 and 99999: 22 occurrences
    invalid literal for int() with base 10: '': 44 occurrences
    Invalid country: MEXICAN-HISPANIC-SPANISH ORIG, SPC 1ORIGIN-MOREAN-CUBAN-BAYESIAN-DONUT SPARROWHEADS-SI AM-PRT-LTH-OTHER-CARA CAB ROLLERS-TURCO: 1 occurrences
    invalid literal for int() with base 10: '3204>': 2 occurrences
    Invalid country: MEXICOFemale : 1 occurrences
    Invalid country: MEXICO: 95 occurrences
    Invalid country: MEXICAN-HISPANIC: 1 occurrences
    invalid literal for int() with base 10: '3201>': 2 occurrences
    invalid literal for int(

...

  Model mistral at Temp 5 - Error Rate: 10.22%
  Error Distribution Summary for Model mistral at Temp 5:
    Invalid occupation: Exectutive: 15 occurrences
    Invalid relationship: Other: 23 occurrences
    Invalid occupation: Exectutive-managerial: 15 occurrences
    Invalid occupation: Execthinktank: 2 occurrences
    invalid literal for int() with base 10: '15000<': 1 occurrences
    Invalid marital status: Married-spouse: 21 occurrences
    Input must have exactly 13 fields separated by |: 211 occurrences
    Invalid occupation: Exectec-managerial: 30 occurrences
    Invalid occupation: Exect-managerial: 40 occurrences
    Invalid occupation: Executive: 16 occurrences
    Invalid occupation: Executive-managerial: 26 occurrences
    Invalid occupation: Execthys-managerial: 1 occurrences
    Invalid occupation: Exectemp-managerial: 13 occurrences
    Invalid education: Professional-school: 1 occurrences
    Invalid occupation: Exectechnicalmanagerial: 7 occurrences
    Invalid occup

Generating Samples for mistral with Temp 10: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [11:48:57<00:00,  8.51s/it]


  Model mistral at Temp 10 - Error Rate: 10.88%
  Error Distribution Summary for Model mistral at Temp 10:
    Input must have exactly 13 fields separated by |: 194 occurrences
    Invalid occupation: Exectemp-managerial: 18 occurrences
    Invalid occupation: Exectutive-managerial: 15 occurrences
    Invalid occupation: Executive: 24 occurrences
    Invalid occupation: Executive-managerial: 30 occurrences
    Invalid relationship: Other: 37 occurrences
    Invalid occupation: Execthespetiality: 1 occurrences
    Invalid marital status: Married-spouse: 21 occurrences
    Invalid occupation: Exectutive: 21 occurrences
    Invalid occupation: Execthic-managerial: 1 occurrences
    Invalid occupation: Execthicalspecialty: 1 occurrences
    Capital loss must be between 0 and 99999: 2 occurrences
    Invalid occupation: Exect-managerial: 40 occurrences
    Invalid occupation: Exectechnicalsupportmanager: 3 occurrences
    Invalid occupation: Exectechnical: 11 occurrences
    Invalid occupat

Generating Samples for mistral with Temp 15: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [5:44:32<00:00,  4.13s/it]

  Model mistral at Temp 15 - Error Rate: 12.00%
  Error Distribution Summary for Model mistral at Temp 15:
    Invalid relationship: Other: 52 occurrences
    Input must have exactly 13 fields separated by |: 225 occurrences
    invalid literal for int() with base 10: '15000<': 1 occurrences
    Invalid occupation: Exectec-managerial: 40 occurrences
    Invalid occupation: Executive: 26 occurrences
    Invalid occupation: Exectech-managerial: 24 occurrences
    Invalid occupation: Execthehelp-managerial: 1 occurrences
    Invalid occupation: Exectutive-managerial: 17 occurrences
    Invalid occupation: Exect-managerial: 37 occurrences
    Invalid occupation: Executive-managerial: 18 occurrences
    Invalid occupation: Exectechnicalsupport: 3 occurrences
    Invalid occupation: Exectemp-managerial: 13 occurrences
    invalid literal for int() with base 10: '30000<': 1 occurrences
    Invalid education: Prof-specialty: 16 occurrences
    Invalid occupation: Exectech-support: 6 occurrence

# Process and Evaluate results

In [35]:
def preprocess(df):
    # Correct column names
    df.columns = df.columns.str.lower().str.replace('-', '_')

    # Clean the income column by stripping spaces and removing periods
    df['target'] = df['target'].str.replace('.', '', regex=False).str.strip()
    
    # Encode '>50K' as 1 and '<=50K' as 0
    df['target'] = df['target'].map({'>50K': 1, '<50K': 0})

    # The dataset has native class for missing data: "?"
    df = df.fillna('?')

    # One-Hot Encoding for categorical variables
    df = pd.get_dummies(df, columns=['relationship', 'marital_status', 'race', 'sex'], drop_first=True)
    df.columns = df.columns.str.lower().str.replace('-', '_') # lower case and use _ instead of -
    
    # Integer Encoding for workclass, occupation, and native-country
    encoder = LabelEncoder()

    # col name missmatches
    df.rename(columns={'country': 'native_country', 'education': 'education_num', 'hours_weekly': 'hours_per_week'}, inplace=True)
    
    # Apply integer encoding
    df['workclass'] = encoder.fit_transform(df['workclass'])
    df['occupation'] = encoder.fit_transform(df['occupation'])
    df['native_country'] = encoder.fit_transform(df['native_country'])
    df['education_num'] = encoder.fit_transform(df['education_num'])
        
    # Convert boolean columns to integers
    df = df.astype(int)

    # HOT ENCODING ISSUE IF YOU MISS CERTAIN VALUES THE COLUMN WONT BE GENERATED
    # Get the columns in X_train_base that are missing in df
    missing_cols = set(X_train_base.columns) - set(df.columns)
    
    # Add the missing columns to df with 0 as the default value
    for col in missing_cols:
        df[col] = 0
    
    # REORDER COLUMNS
    ordered_columns = [col for col in X_train_base.columns if col != 'target']  
    
    # Add 'target' at the end
    ordered_columns += ['target']  
    
    # Reorder
    df = df[ordered_columns]
    
    # List of columns to scale (all columns except 'income')
    numeric_cols = df.columns[df.columns != 'target']
  
    # Initialize the scaler
    scaler = StandardScaler()
    
    # Scale the numerical features and update the DataFrame
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

    return df

In [36]:
# Define the column structure
columns = [
    "Age", "Sex", "Country", "Marital-Status", "Occupation", "Workclass",  "Education",
    "Race", "Relationship", "Hours-Weekly", "Capital-Gain", "Capital-Loss", "Target"
]

for model in models:
    for temp in temperatures:
        # Get the list of pipe-separated strings
        rows = results[model][temp]
        
        # Split each string into a list of values
        split_rows = [row.split('|') for row in rows]
        
        # Create DataFrame
        df = pd.DataFrame(split_rows, columns=columns)
        df = preprocess(df)

        # Split the data into X and y
        X_train = df.drop('target', axis=1)
        y_train = df['target']
        
        # Print the shapes of the sets
        print("Model: " + model + ", Temperature = " + str(temp))
        print(f"Training Set: X_train shape = {X_train.shape}, y_train shape = {y_train.shape}")
        print(f"Test Set: X_test shape = {X_test_base.shape}, y_test shape = {y_test_base.shape}") # We use the test data from base!
        print()
        
        temp_result = []
        for name, ml_model in ml_models:
            # Train the model
            ml_model.fit(X_train, y_train)
        
            # Predict on test set
            y_test_pred_base = ml_model.predict(X_test_base)

            # Store results for the model
            model_results = {
                'Model': name,
                'Test Accuracy': accuracy_score(y_test_base, y_test_pred_base),
                'Test Precision': precision_score(y_test_base, y_test_pred_base),
                'Test Recall': recall_score(y_test_base, y_test_pred_base),
                'Test F1-Score': f1_score(y_test_base, y_test_pred_base)
            }
        
            temp_result.append(model_results)

        # Convert results to a pandas DataFrame
        results_df = pd.DataFrame(temp_result).set_index("Model")

        # Calculate difference dfs (absolute and % difference)
        result_diff = results_df - base_results
        result_pct_diff = (result_diff / base_results) * 100

        print("Results Data Synthesis:")
        display(results_df)
        print("Absolute Difference with base:")
        display(result_diff)
        print("Percentage Difference with base (%):")
        display(result_pct_diff)
        print()
        euclidean(df)
        pca(df)
        print()

Model: llama3.1, Temperature = 5
Training Set: X_train shape = (3675, 24), y_train shape = (3675,)
Test Set: X_test shape = (9769, 24), y_test shape = (9769,)

Results Data Synthesis:


,Test Accuracy,Test Precision,Test Recall,Test F1-Score
Model,,,,
Logistic Regression,0.293582,0.237347,0.872187,0.373149
Random Forest,0.391545,0.241836,0.713800,0.361272
SVM,0.268605,0.241918,0.953291,0.385905
KNN,0.416931,0.225472,0.582590,0.325118
Gradient Boosting,0.264920,0.237660,0.928238,0.378430


Absolute Difference with base:


,Test Accuracy,Test Precision,Test Recall,Test F1-Score
Model,,,,
Logistic Regression,-0.516941,-0.399610,0.374522,-0.185611
Random Forest,-0.422049,-0.413578,0.235669,-0.191637
SVM,-0.534446,-0.403592,0.547346,-0.112531
KNN,-0.375064,-0.366866,0.142675,-0.179755
Gradient Boosting,-0.539973,-0.365844,0.372399,-0.200262


Percentage Difference with base (%):


...

Absolute Difference with base:


,Test Accuracy,Test Precision,Test Recall,Test F1-Score
Model,,,,
Logistic Regression,-0.058655,-0.426872,-0.487049,-0.538550
Random Forest,-0.064797,-0.511528,-0.469639,-0.536871
SVM,-0.044119,-0.645510,-0.405945,-0.498436
KNN,-0.074214,-0.316669,-0.335032,-0.352920
Gradient Boosting,-0.053537,-0.473504,-0.550318,-0.568101


Percentage Difference with base (%):


,Test Accuracy,Test Precision,Test Recall,Test F1-Score
Model,,,,
Logistic Regression,-7.236676,-67.017524,-97.866894,-96.383032
Random Forest,-7.964268,-78.046692,-98.223801,-97.099255
SVM,-5.493945,-100.000000,-100.000000,-100.000000
KNN,-9.370557,-53.460791,-76.158301,-69.902697
Gradient Boosting,-6.651405,-78.459129,-99.006875,-98.169900



Average pairwise distance: 6.076252105833049
Explained variance ratio: [0.15446225 0.07023303 0.06447521]
Cumulative explained variance: 0.28917048568284953



In [64]:
# Get unique classes and counts
unique_classes, counts = np.unique(y_test, return_counts=True)
class_balance = dict(zip(unique_classes, counts))

print("Class Balance (Counts):", class_balance)
print("Class Balance (%):", {cls: count / len(y_test) * 100 for cls, count in class_balance.items()})

Class Balance (Counts): {0: 7414, 1: 2355}
Class Balance (%): {0: 75.89313133381104, 1: 24.106868666188966}
